In [ ]:
import pandas as pd
from pathlib import Path
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from citipy import citipy
import hvplot.pandas
from api_keys_mm import weather_api_key
from api_keys_mm import geoapify_key
import os

In [ ]:
drugDataDF = pd.read_csv(("drug_deaths.csv"))
pd.set_option('display.max_columns', None)
drugDataDF.head()

In [ ]:
drugColDF = drugDataDF[['ID', 'Date', 'DateType', 'Age', 'Sex', 'Race',
       'ResidenceCity', 'ResidenceCounty', 'ResidenceState', 'DeathCity',
       'InjuryState', 'Heroin', 'Cocaine', 'Fentanyl', 'Fentanyl_Analogue',
       'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine',
       'Methadone', 'Amphet', 'Tramad', 'Morphine_NotHeroin', 'Hydromorphone',
       'Other', 'OpiateNOS', 'AnyOpioid', 'DeathCityGeo']]
drugColDF.head()

In [ ]:
drugLocDF = drugColDF.loc[drugColDF["ResidenceState"] == "CT"]
drugLocDF = drugLocDF.loc[drugColDF["Sex"] != "Unknown"]
drugLocDF = drugLocDF.loc[drugColDF["Race"] != "Unknown"]

drugLocDF["Fentanyl"] = drugLocDF["Fentanyl"].replace("1-A","1")
drugLocDF["Morphine_NotHeroin"] = drugLocDF["Morphine_NotHeroin"].replace(["1ES","STOLE MEDS", "NO RX BUT STRAWS","PCP NEG"],0)
drugLocDF["Morphine_NotHeroin"] = drugLocDF["Morphine_NotHeroin"].astype(int)
drugLocDF["Fentanyl_Analogue"] = drugLocDF["Fentanyl_Analogue"].astype(int)
drugLocDF["Fentanyl"] = drugLocDF["Fentanyl"].astype(int)


drugLocDF.head()

In [ ]:
drugLocDF.to_csv("drug_deaths_clean.csv")

In [ ]:
Heroin = drugLocDF["Heroin"].sum()
Cocaine = drugLocDF["Cocaine"].sum()
Fentanyl = drugLocDF["Fentanyl"].sum()
Fentanyl_Analogue = drugLocDF["Fentanyl_Analogue"].sum()
Oxycodone = drugLocDF["Oxycodone"].sum()
Oxymorphone = drugLocDF["Oxymorphone"].sum()
Ethanol = drugLocDF["Ethanol"].sum()
Hydrocodone = drugLocDF["Hydrocodone"].sum()
Benzodiazepine = drugLocDF["Benzodiazepine"].sum()
Methadone = drugLocDF["Methadone"].sum()
Amphet = drugLocDF["Amphet"].sum()
Tramad = drugLocDF["Tramad"].sum()
Morphine_NotHeroin = drugLocDF["Morphine_NotHeroin"].sum()
Hydromorphone = drugLocDF["Hydromorphone"].sum()
OpiateNOS = drugLocDF["OpiateNOS"].sum()



drug_stats = pd.DataFrame({"Heroin":[Heroin],
                          "Cocaine":[Cocaine],
                          "Fentanyl":[Fentanyl],
                          "Oxycodone":[Oxycodone],
                          "Oxymorphone":[Oxymorphone],
                          "Ethanol":[Ethanol],
                          "Hydrocodone":[Hydrocodone],
                          "Benzodiazepine":[Benzodiazepine],
                          "Methadone":[Methadone],
                          "Amphet":[Amphet],
                          "Tramad":[Tramad],
                          "Morphine":[Morphine_NotHeroin],
                          "Hydromorphone":[Hydromorphone],
                          "OpiateNOS":[OpiateNOS],
                         })
drug_stats

In [ ]:

plot_pandas = drug_stats.plot(kind="bar", linewidth=1, figsize=(16, 10)) 
plt.xlabel("Drugs", fontsize = 25)
plt.ylabel("Death Toll Count", fontsize = 25)
plt.title("Death Total Across CT", fontsize = 35)

plt.savefig("Images/DeathTotalBarGraph.png")

plt.show()

In [ ]:
# import drug_death_clean.csv 
drug1DF = pd.read_csv(("../Data Cleaning/drug_deaths_clean.csv"))
pd.set_option('display.max_columns', None)
drugDF = drug1DF[['ID', 'Date', 'DateType', 'Age', 'Sex', 'Race',
       'ResidenceCity', 'ResidenceCounty', 'ResidenceState', 'DeathCity',
       'InjuryState', 'Heroin', 'Cocaine', 'Fentanyl', 'Fentanyl_Analogue',
       'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine',
       'Methadone', 'Amphet', 'Tramad', 'Morphine_NotHeroin', 'Hydromorphone',
       'Other', 'OpiateNOS', 'AnyOpioid', 'DeathCityGeo']]
drugDF.head()

In [ ]:
# break dates down into lists
dates = []
year = []
month = []
day = []

for x in range(len(drugDF.index)):
    dates.append(drugDF['Date'][x].split()[0])
    year.append(drugDF['Date'][x].split()[0].split('/')[2])
    month.append(drugDF['Date'][x].split()[0].split('/')[0])
    day.append(drugDF['Date'][x].split()[0].split('/')[1])
    #print(f'{month[x]} {day[x]} {year[x]} | {dates[x]}')

In [ ]:
# create a data frame for new date variables
dateDF = pd.DataFrame({'Date':dates,
                       'Month':month,
                       'Day':day,
                       'Year':year,
                       'Season':''})
dateDF

In [ ]:
# assign seasons to date dataframe
for x in range(len(dateDF.index)):
    if dateDF['Month'][x] == '03' or dateDF['Month'][x] == '04' or dateDF['Month'][x] == '05':
        dateDF['Season'][x] = 'Spring'
    elif dateDF['Month'][x] == '06' or dateDF['Month'][x] == '07' or dateDF['Month'][x] == '08':
        dateDF['Season'][x] = 'Summer'
    elif dateDF['Month'][x] == '09' or dateDF['Month'][x] == '10' or dateDF['Month'][x] == '11':
        dateDF['Season'][x] = 'Autumn'
    elif dateDF['Month'][x] == '12' or dateDF['Month'][x] == '01' or dateDF['Month'][x] == '02':
        dateDF['Season'][x] = 'Winter'
    else:
        dateDF['Season'][x] = 'N/A'
dateDF

In [ ]:
# create dataframe for each season
SpringDF = dateDF.loc[dateDF['Season'] == 'Spring']
SummerDF = dateDF.loc[dateDF['Season'] == 'Summer']
AutumnDF = dateDF.loc[dateDF['Season'] == 'Autumn']
WinterDF = dateDF.loc[dateDF['Season'] == 'Winter']

In [ ]:
SpringDF['Year'] = pd.to_numeric(SpringDF['Year'])
SummerDF['Year'] = pd.to_numeric(SummerDF['Year'])
AutumnDF['Year'] = pd.to_numeric(AutumnDF['Year'])
WinterDF['Year'] = pd.to_numeric(WinterDF['Year'])
SpringDF.sort_values(by=['Year'])
SummerDF.sort_values(by=['Year'])
AutumnDF.sort_values(by=['Year'])
WinterDF.sort_values(by=['Year'])

In [ ]:
#assign color values
purple = clr.to_rgba('darkorchid', alpha=0.53)
lBlue = clr.to_rgba('deepskyblue', alpha=0.73)
dBlue = clr.to_rgba('mediumblue', alpha=0.75)
gold = clr.to_rgba('goldenrod', alpha=0.9)
colors = [purple, lBlue, gold, dBlue]
x = np.arange(2014.5,2019,1)

#plot and label histogram
plt.figure(figsize=(6.5,5.5))
plt.hist([SpringDF['Year'], SummerDF['Year'], AutumnDF['Year'], WinterDF['Year']],
         label=['Spring', 'Summer', 'Autumn', 'Winter'], color = colors,bins = x)

plt.xticks([2015, 2016, 2017, 2018])
plt.ylim(-0, 300)
plt.title("Drug Deaths in each Season Per Year in Connecticut")
plt.xlabel("Seasons by Year")
plt.ylabel("Drug Deaths")
plt.legend(loc="upper left")
plt.savefig("Images/SeasonsHistogram.png")

In [ ]:
#create new DF with the count of each category
Death_by_race_count =drugLocDF["Race"].value_counts()
Death_by_race_count

In [ ]:
#Create Race DF
#Set index and rename column value 
Death_by_race_count = drugLocDF["Race"].value_counts().rename_axis("sub_race_values").reset_index(name='counts')
Death_by_race_count

In [ ]:
#Combined_other_race = race_df.replace(Asian Indian, np.Other, inplace = True)
Combined_other_race_df= Death_by_race_count
Combined_other_race_df['sub_race_values'] = Combined_other_race_df['sub_race_values'].replace(['Asian Indian','Asian, Other','Other','Chinese','Hawaiian','Hispanic, Black','Hispanic, White' ],['Asian and Some Other','Asian and Some Other','Asian and Some Other','Asian and Some Other','Asian and Some Other','Hispanic','Hispanic'])
combined_race_df = Combined_other_race_df.groupby('sub_race_values',as_index = False).agg('sum')

#df.reset.index(as_index = False).groupby(as_index = False).sum()
print(combined_race_df)
combined_race_df

In [ ]:
#taking the total of counts column to get the value out of the percentage
total_death_counts = combined_race_df.counts.sum()
total_death_counts

In [ ]:
# to add the values along with percentage we have to do reverse calculations using a function
#
def my_autopct_function(prct_value):
    (prct_value)
    return'{:.0f}%\n{:.0f} deaths'.format(prct_value, total_death_counts*prct_value/100)

In [ ]:
#set labels and values to display in pie chart
labels = combined_race_df.sub_race_values
values = combined_race_df.counts
explode = [0.1,0,0.1,0.1,] # explode is = to the number of  index from the count of the key value


In [ ]:
#visualize the pie chart
fig = plt.figure()
ax =fig.add_axes([0,0,2,2])
ax.axis('equal')
ax.pie(values, autopct = my_autopct_function, explode = explode,
       textprops={'fontsize':11, 'weight': 'bold', 'color': 'white'},
       wedgeprops = {'edgecolor': 'black'},shadow=True)
plt.title(f" Drug related deaths in Connecticut by Race\n\nTotal Deaths (2015-2018): {total_death_counts}")
plt.legend(labels, title = "Legend", loc="best")
plt.savefig("Images/racechart.png", bbox_inches = 'tight')
plt.show()
#autopct function returns List of strings used to Label the wedge with their numerical value
#explode - List the margin in the sequence of data 


In [ ]:
#read cleaned data from CSV file in folder
clean_data = pd.read_csv(("drug_deaths_clean.csv"))
pd.set_option('display.max_columns', None)
clean_data.info()
clean_data.head()

In [ ]:
#I noticed in code below when I tried to combine drug types, Fentanyl column is a string. See output below for why
#Turns out several drug types are strings. See df.info() above for which ones
print(clean_data["Fentanyl"].value_counts())
clean_data["Morphine_NotHeroin"].value_counts()

In [ ]:
#replace weird value with 1 so I cann add columns together
clean_data["Fentanyl"] = clean_data["Fentanyl"].replace(["1-A", " "],"1")
clean_data["Morphine_NotHeroin"] = clean_data["Morphine_NotHeroin"].replace(["1ES","STOLE MEDS", " ", "NO RX BUT STRAWS","PCP NEG"],0)


clean_data["Fentanyl"].value_counts()
clean_data["Morphine_NotHeroin"].value_counts()

In [ ]:
#reformat all changed string columns as integers
clean_data["Fentanyl"] = clean_data["Fentanyl"].astype(int)
clean_data["Morphine_NotHeroin"] = clean_data["Morphine_NotHeroin"].astype(int)
clean_data["Fentanyl_Analogue"] = clean_data["Fentanyl_Analogue"].astype(int)
clean_data.head()

In [ ]:
clean_data["Total Drug Deaths"] = clean_data["Heroin"] + clean_data["Cocaine"] + clean_data["Fentanyl"] + clean_data["Fentanyl_Analogue"] + clean_data["Oxycodone"] + clean_data["Oxymorphone"] + clean_data["Ethanol"] + clean_data["Hydrocodone"] + clean_data["Benzodiazepine"] + clean_data["Methadone"] + clean_data["Amphet"] + clean_data["Tramad"] + clean_data["Morphine_NotHeroin"]
clean_data.head()

In [ ]:
#Use weather API to extract latitude and longitude data for each city to use for HVPLOT
#test weather API call for 1 city to make sure it works

# Set the API base URL
base_url = "http://api.openweathermap.org/data/2.5/weather?"


# Create endpoint URL with each city, testing it with 2nd city in list
city_url = base_url + "appid=" + weather_api_key + "&q=" + "Middletown"
response = requests.get(city_url).json()
pprint(response)

In [ ]:
#Grouping by city to condense rows and make for loop below go much faster. 3425 rows -> 207 rows
groupby_city = clean_data.groupby(["DeathCity"]).sum()


In [ ]:
#use for loop to loop through cities and extract latitude and longitude for each city

#define empty list to append lat and lng data into
lat = []
lng = []

#loop through city list and extract lat and lng for each city, and append them to their respective list. groupby_city.index is the city name
for city in groupby_city.index:
    print(city)
    
    #define base URL
    base_url = "http://api.openweathermap.org/data/2.5/weather?"


    # Create endpoint URL with each city, testing it with 2nd city in list
    city_url = base_url + "appid=" + weather_api_key + "&q=" + city
    response = requests.get(city_url).json()
    
    #need try/except statment if not lat/lng data is available.
    try:
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        
    #need to fill in null value in lat & lng lists if no data is there because the list length has to be the same as DF length to add the lists to the DF
    except:
        print("No coordinates available")
        #lat.append(np.nan) #putting in null values stops plot from running. Put in 0 and filter them out later
        lat.append(0)
        lng.append(0)

groupby_city["Latitude"] = lat
groupby_city["Longitude"] = lng
groupby_city

In [ ]:
# Configure the map plot
map1 = groupby_city.hvplot.points("Longitude","Latitude", geo=True, tiles="OSM", 
                                  frame_width = 700, frame_height = 500,
                                  size = "Total Drug Deaths")
map1

In [ ]:
#filter out areas that are not Connecticut.
groupby_city["Total Drug Deaths"].sum() #there are 7857 total deaths. Use as reference to see how many are cut out

#filter out cities outside this region. Weather.py picked up some cities with the same name in another part of the world
groupby_city_filtered = groupby_city.loc[(groupby_city["Latitude"]>40) & 
                                         (groupby_city["Latitude"]<43) & 
                                         (groupby_city["Longitude"]>-80)]

groupby_city_filtered["Total Drug Deaths"].sum() #narrowed it from 7857 cities down to 6080 cities
groupby_city_filtered.reset_index()

In [ ]:
#need to copy death city column to color points by city and lebel them because DeathCity is the index and I can't figure out how to color by index
groupby_city_filtered["DeathCity"] = groupby_city_filtered.index

In [ ]:
map2 = groupby_city_filtered.hvplot.points("Longitude","Latitude", geo=True, tiles="OSM", 
                                  frame_width = 700, frame_height = 500,
                                  size = "Total Drug Deaths", 
                                  color = "DeathCity")
map2

In [ ]:
colors=['green', 'red']
explode = (0.1, 0)
gender_data = drugLocDF["Sex"].value_counts()
plt.title("Female vs. Male")
gender_data.plot.pie(colors=colors, autopct="%1.1f%%", explode=explode, shadow=True, startangle=100)
plt.axis("equal")
plt.savefig("Images/FemaleVsMalePie.png")
plt.show()

In [ ]:
#read drug overdose data in 
drug_deaths = pd.read_csv(("drug_deaths_clean.csv"))
drug_deaths = drug_deaths.loc[(drug_deaths["Fentanyl"] != " ")]
drug_deaths["Fentanyl"] = drug_deaths["Fentanyl"].replace(["1-A"],"1")

print(drug_deaths.shape)
print(drug_deaths.columns)
drug_deaths.head()

In [ ]:
fent_cols = ["Age", "Fentanyl", "Fentanyl_Analogue"]
drug_deaths[fent_cols].head()

In [ ]:
drug_deaths["Fentanyl"].value_counts()

In [ ]:
age_df = drug_deaths.groupby("Age").count()
age_df["Age"] = age_df.index

In [ ]:
age_df.plot(kind="scatter", x="Age", y="Fentanyl", color="blue")
plt.title("Age vs. Fentanyl Overdoses")

plt.savefig("Images/AgeVsFentanyl.png")